### Secure SEC files

In [13]:
import os
import requests
import pandas as pd
from collections import namedtuple
from itertools import product
from azure.storage.blob.blockblobservice import BlockBlobService
import urllib.request
from keystore import get_key

from fastprogress import master_bar, progress_bar

In [14]:
save_dir = '/home/datawrestler/data/secfiles'

In [15]:
class Blob(object):
    
    def __init__(self, account_name='generalstorageblob2019',
                account_key=None):
        
        if account_key is None:
            account_key = get_key(key_type='BLOB')
        
        self.service = BlockBlobService(account_name=account_name,
                                       account_key=account_key)
        
        self.azure_store = {
            '10-K': 'annual',
            '10-K/A': 'annual',
            '10-Q': 'quarterly',
            '10-Q/A': 'quarterly',
            'DEF 14A': 'proxy',
            'DEFA14A': 'proxy'
        }
        
    def write_blob(self, ftype, fname, fpath):
        
        blobname = self.azure_store[ftype]
        
        if not self.service.exists(blobname):
            self.service.create_container(blobname)
            
        
        self.service.create_blob_from_path(blobname, 
                                          fname, 
                                          fpath)
        
            

In [ ]:
from azure.storage.blob import BlockBlobService, PublicAccess
from azure.storage.blob.models import Blob

def run_sample():
    block_blob_service = BlockBlobService(account_name='your_name', account_key='your_key')
    container_name ='t1s'

    block_blob_service.copy_blob(container_name,'remoteURL.pdf','https://media.readthedocs.org/pdf/azure-storage/v0.20.3/azure-storage.pdf')


# Main method.
if __name__ == '__main__':
    run_sample()

In [22]:
blob = Blob()
name = '0000002178-13-000014.txt'
blob.service.make_blob_url('annual', name)

'https://generalstorageblob2019.blob.core.windows.net/annual/0000002178-13-000014.txt'

In [45]:

sec_files = '/home/datawrestler/data/sec/annual'
files = os.listdir(sec_files)


In [ ]:
from tqdm.auto import tqdm
for f in tqdm(files, total=len(files)):
    block_blob_service.create_blob_from_path(annual_name,
                                            f,
                                            os.path.join(sec_files, f))

In [ ]:
# full index of SEC files
full_index_url = 'https://www.sec.gov/Archives/edgar/full-index/{year}/QTR{quarter}/form.idx'
base_url = 'https://www.sec.gov/Archives/{file_url}'

get_form = lambda x: str(requests.get(x).content)
formIDX = requests.get(full_index_url.format(year=2015, quarter=4)).content

In [ ]:
def read_seen_files():
    master_record = pd.read_csv(os.path.join(save_dir, 'master_record.txt'), sep='\t', header=None
                                 )
    
    master_record.columns = ['Form Type', 
                        'Company Name',
                        'CIK',
                        'Date', 
                        'URL',
                        'download_path']

    seen_files = master_record['URL'].unique().tolist()
    
    return seen_files

seen_files = read_seen_files()

In [ ]:
clean_lines = lambda x: [line.strip() for line in x.split('  ') if line.strip() != '']

def clean_form_lines(form):
    # split on newline char
    all_rows = []
    all_lines = form.split('\\n')
    for line in all_lines[1:]:
        line = clean_lines(line)
        if len(line) == 5:
            row = convert_line_to_row(line)
            all_rows.append(row)
    # drop header line ['Form Type', 'Company Name', 'CIK', 'Date Filed', 'File Name']
    return pd.concat(all_rows).iloc[1:]

def convert_line_to_row(line):
    return pd.DataFrame({'Form Type': line[0], 
                        'Company Name': line[1],
                        'CIK': line[2],
                        'Date Filed': line[3],
                        'File Name': line[4]}, index=[0])

In [ ]:
def check_dir(file_type):
    # create dir for specified file type if it doesnt exist in save_dir
    path = os.path.join(save_dir, file_type)
    if not os.path.exists(path):
        os.makedirs(path)
    return path

In [ ]:
def write_master_record(line):
    master_record = os.path.join(save_dir, 'master_record.txt')
    with open(master_record, 'a') as file:
        file.write("\n{}".format(line)) # could be any text, appended @ the end of file
    file.close()

def download_sec_file(row, blob):
    try:
        form_type = row['Form Type']
        file_url = row['File Name']
        form_name = file_url.split('/')[-1]
        full_file_path = os.path.join(cache_dir, form_name)
        urllib.request.urlretrieve(base_url.format(file_url=file_url), full_file_path)
        blob.write_blob(form_type, form_name, full_file_path)
        # remove file from cache
        os.remove(full_file_path)
        return full_file_path
    except:
        return None
    


In [ ]:
formdf = clean_form_lines(get_form(full_index_url.format(year=2015, quarter=4)))
formdf.head()

In [ ]:
def subset_formdf(formdf, wanted_forms=None, seen_files=None):
    if wanted_forms is None:
        wanted_forms = [
            '10-K',
            '10-K/A',
            '10-Q', 
            '10-Q/A',
            'DEF 14A',
            'DEFA14A',
        ]
        
    
    out = formdf.loc[formdf['Form Type'].isin(wanted_forms)]
    
    if seen_files is not None:
        out = formdf.loc[~formdf['File Name'].isin(seen_files)]
    
    return out


In [46]:
quarters = list(range(1, 5))
years = list(reversed(list(range(2010, 2019))))
cache_dir = '.'

In [51]:
time = list(product(quarters, years))

In [52]:
from tqdm.auto import tqdm
for timeslice in tqdm(time, total=len(time)):
    qtr = timeslice[0]
    yr = timeslice[1]

In [ ]:


mb = master_bar(list(product(quarters, years)))

alldf = []

for qtr, yr in mb:
    mb.first_bar.comment = 'Year - {} Quarter - {}'.format(yr, qtr)
    form_url = full_index_url.format(year=yr, quarter=qtr)
    form = get_form(form_url)
    formdf = clean_form_lines(form)
    # subset to correct form types
    formdf = subset_formdf(formdf, seen_files=seen_files)
    all_paths = []
    for index, row in progress_bar(formdf.iterrows(), total=formdf.shape[0], parent=mb):
        full_file_path = download_sec_file(row)
        form_type = row['Form Type']
        co_name = row['Company Name']
        cik = row['CIK']
        date_filed = row['Date Filed']
        file_name = row['File Name']
        line = '\t'.join([form_type, co_name, cik, date_filed, file_name, full_file_path])
        write_master_record(line)
        all_paths.append(full_file_path)
        mb.child.comment = 'Downloaded: {}'.format(co_name)

    formdf['full_file_path'] = all_paths
    alldf.append(formdf)

In [ ]:
from sqlalchemy import create_engine

In [2]:
import pyodbc

In [9]:
from sqlalchemy import create_engine
import urllib

conn = get_key(key_type='SQL')
params = urllib.parse.quote_plus(conn)
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)


In [10]:
import pandas as pd
import numpy as np

# get written log
log = pd.read_sql('select * from dbo.seclog', engine)

In [11]:
log

,form_type,co_name,cik,date_filed,file_name,success
0,10-K,"1347 Property Insurance Holdings, Inc.",1591890,2018-03-26,edgar/data/1591890/0001493152-18-003887.txt,None
1,10-K,"1347 Property Insurance Holdings, Inc.",1591890,2018-03-26,edgar/data/1591890/0001493152-18-003887.txt,None
2,10-K,"1347 Property Insurance Holdings, Inc.",1591890,2018-03-26,edgar/data/1591890/0001493152-18-003887.txt,1
3,10-K,"1347 Property Insurance Holdings, Inc.",1591890,2018-03-26,edgar/data/1591890/0001493152-18-003887.txt,1
4,10-K,"1347 Property Insurance Holdings, Inc.",1591890,2018-03-26,edgar/data/1591890/0001493152-18-003887.txt,1
5,10-K,"1347 Property Insurance Holdings, Inc.",1591890,2018-03-26,edgar/data/1591890/0001493152-18-003887.txt,1
6,10-K,"1347 Property Insurance Holdings, Inc.",1591890,2018-03-26,edgar/data/1591890/0001493152-18-003887.txt,1
7,10-K,1ST CONSTITUTION BANCORP,1141807,2018-03-19,edgar/data/1141807/0001141807-18-000005.txt,1
8,10-K,1ST SOURCE CORP,34782,2018-02-16,edgar/data/34782/0000034782-18-000033.txt,1
9,10-K,1st FRANKLIN FINANCIAL CORP,38723,2018-03-30,edgar/data/38723/0001376474-18-000064.txt,1
